In [36]:
#import packages

import numpy as np
import pandas as pd
import random
import pickle
import itertools
import folium #mapping
import seaborn as sns #color palette
#from colormap import rgb2hex #chnage RGB codes to hex codes

In [37]:
## USE THIS CODE TO IMPORT DATA

file_name = '../data/results/results_pooled.pkl' ##results_toydata_uniform.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [38]:
#convert to dataframe

df = pd.DataFrame(loaded_list, columns = ['supplier', 'receiver', 'distance', 'volume'])
df.head()

#convert suppliers to list to get rid of weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()
#supplier_split[0:5]


#convert suppliers to lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]

#convert to list of receiver lat and long
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lat

df.head()

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[33.89453583, -118.2737097]",232.975782,27181.7970,36.92375,-120.10412,33.894536,33.894536
1,"[36.92375, -120.10412]","[34.075433000000004, -118.376871]",219.172672,2153.0845,36.92375,-120.10412,34.075433,34.075433
2,"[36.92375, -120.10412]","[34.0733235, -118.3571457]",219.799267,31425.2180,36.92375,-120.10412,34.073324,34.073324
3,"[37.7965, -121.6049]","[35.451214, -119.715509]",192.809931,1062.8875,37.79650,-121.60490,35.451214,35.451214
4,"[37.7965, -121.6049]","[33.94227004, -117.8910835]",337.639164,51838.6120,37.79650,-121.60490,33.942270,33.942270


In [53]:
#create a color palette, repeat it 100 times to see how many things I can visualize at once in the html
palette = ['red', 'green', 'purple', 'orange',  'darkblue',
           'gray','cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
           'black','lightgray','lightred','blue','beige']*100
            #‘lightgreen’] ‘gray’, ‘black’, ‘lightgray’] 'darkred','lightred','blue','beige','white',


#palette

## Map Everything

Shape of dataframe

In [54]:
df.shape

(3961, 8)

In [55]:
#view a record
df[df['supplier_lat'] == 35.399380]

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
2037,"[35.39938, -119.40813]","[38.58910956, -121.63689879999998]",252.127988,194.4,35.39938,-119.40813,38.58911,38.58911


Get counts by supplier

In [56]:
#counts by each supplier
df_counts = df.groupby(['supplier_lat','supplier_lon']).count().reset_index()
df_sorted = df_counts.sort_values('distance')
df_sorted.tail(20)

,supplier_lat,supplier_lon,supplier,receiver,distance,volume,receiver_lat,receiver_lon
105,34.604500,-117.338200,55,55,55,55,55,55
30,33.746392,-117.860447,57,57,57,57,57,57
6,32.802200,-115.540000,58,58,58,58,58,58
17,33.140800,-117.334200,62,62,62,62,62,62
55,33.917500,-118.425800,67,67,67,67,67,67
138,35.418500,-119.011200,68,68,68,68,68,68
216,37.553007,-120.916475,75,75,75,75,75,75
139,35.447350,-119.086520,77,77,77,77,77,77
124,35.092330,-119.407371,85,85,85,85,85,85
215,37.528661,-122.026190,94,94,94,94,94,94


Show the 10 suppliers with the shortest mean distance

In [57]:
#group the dataframe by mean distance
df_grouped = df.groupby(['supplier_lat','supplier_lon']).mean('distance').reset_index()
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,32.567200,-116.944400,230.486435,3526.950000,35.340029,35.340029
1,32.573300,-116.915300,190.998102,4488.900016,34.824889,34.824889
2,32.574756,-116.917302,343.638008,17763.150150,36.462857,36.462857
3,32.696772,-117.145403,278.741208,93.400000,35.977634,35.977634
4,32.718311,-115.111520,314.548641,7102.577804,35.373184,35.373184
...,...,...,...,...,...,...
274,40.736900,-122.322300,412.114237,34.100000,35.172068,35.172068
275,40.741500,-124.210300,540.530921,7715.377280,34.330101,34.330101
276,41.024968,-121.693244,362.885207,1089.381350,35.998523,35.998523
277,41.031300,-121.681700,385.432068,29878.959955,35.796108,35.796108


In [58]:
#sort on distance
df_grouped = df_grouped.sort_values('distance')
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
50,33.894790,-118.164740,5.401900,381.100000,33.816421,33.816421
52,33.910116,-118.221906,13.662237,34860.950000,34.073306,34.073306
79,34.125000,-119.400300,14.223849,170019.900000,34.313742,34.313742
46,33.854967,-118.336907,16.821369,31836.900000,33.943705,33.943705
71,34.074052,-118.437913,17.141839,2866.000000,34.315426,34.315426
...,...,...,...,...,...,...
9,32.876900,-117.166400,470.071890,2865.100003,38.635845,38.635845
24,33.615700,-114.686500,499.613889,15.200000,38.112847,38.112847
18,33.243200,-115.498000,507.282890,391022.000000,38.646900,38.646900
23,33.603440,-114.643390,526.264702,22158.900000,38.672759,38.672759


In [59]:
#cut off at the top 10
df_grouped = df_grouped.reset_index() #10 at the .tail(10) head(10)
df_grouped

,index,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,50,33.894790,-118.164740,5.401900,381.100000,33.816421,33.816421
1,52,33.910116,-118.221906,13.662237,34860.950000,34.073306,34.073306
2,79,34.125000,-119.400300,14.223849,170019.900000,34.313742,34.313742
3,46,33.854967,-118.336907,16.821369,31836.900000,33.943705,33.943705
4,71,34.074052,-118.437913,17.141839,2866.000000,34.315426,34.315426
...,...,...,...,...,...,...,...
274,9,32.876900,-117.166400,470.071890,2865.100003,38.635845,38.635845
275,24,33.615700,-114.686500,499.613889,15.200000,38.112847,38.112847
276,18,33.243200,-115.498000,507.282890,391022.000000,38.646900,38.646900
277,23,33.603440,-114.643390,526.264702,22158.900000,38.672759,38.672759


In [60]:
df_small = pd.DataFrame()

#loop through and get the full dataset. can't group by the lat/long because it's some unhashable type.
for i in range(0,df_grouped.shape[0]):
    this_lat = df_grouped['supplier_lat'][i]
    this_lon = df_grouped['supplier_lon'][i]
    receivers = df[(df['supplier_lat'] == this_lat) & (df['supplier_lon'] == this_lon)]
    df_small = df_small.append(receivers)
    
df_small.shape

(3961, 8)

In [61]:
#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers.shape
#unique_suppliers['supplier_lon'][0]

(279, 2)

In [62]:
df

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[33.89453583, -118.2737097]",232.975782,27181.79700,36.92375,-120.10412,33.894536,33.894536
1,"[36.92375, -120.10412]","[34.075433000000004, -118.376871]",219.172672,2153.08450,36.92375,-120.10412,34.075433,34.075433
2,"[36.92375, -120.10412]","[34.0733235, -118.3571457]",219.799267,31425.21800,36.92375,-120.10412,34.073324,34.073324
3,"[37.7965, -121.6049]","[35.451214, -119.715509]",192.809931,1062.88750,37.79650,-121.60490,35.451214,35.451214
4,"[37.7965, -121.6049]","[33.94227004, -117.8910835]",337.639164,51838.61200,37.79650,-121.60490,33.942270,33.942270
...,...,...,...,...,...,...,...,...
3956,"[37.63093, -120.92309]","[35.461627, -119.73641299999998]",163.498302,408.06975,37.63093,-120.92309,35.461627,35.461627
3957,"[37.63093, -120.92309]","[35.470821, -119.012806]",182.962729,3891.18160,37.63093,-120.92309,35.470821,35.470821
3958,"[37.63093, -120.92309]","[35.618133, -119.724226]",153.952694,922.98870,37.63093,-120.92309,35.618133,35.618133
3959,"[37.63093, -120.92309]","[35.23464, -119.615311]",180.576198,3949.09750,37.63093,-120.92309,35.234640,35.234640


In [63]:
#determine the maximum volume - this will be used to scale the line weight
volumes = df['volume']
median_volume = np.median(volumes)
median_volume

volumes = df['volume']
quantiles = np.quantile(volumes, [0.2,0.4,0.6,0.8])

In [64]:
quantiles

array([ 402.95276, 1333.0052 , 3688.6855 , 9628.3471 ])

In [73]:
#create map

#center the map in Cali
#mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=20 )
receiver_count = 0
 
#plot all the suppliers 
latlon = df_small['supplier']
mapit = folium.Map( location=[37.3427, -119.2244], zoom_start=6)

#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#loop through suppliers, paint all the receivers in the same color
for i in range(0,unique_suppliers.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i] ], fill=True, 
                        fill_color=palette[i], color=palette[i],  radius=5 ).add_to( mapit ) #'
    
    #find all receivers associated with the supplier late and lon, filter the df
    this_lat = unique_suppliers['supplier_lat'][i]
    this_lon = unique_suppliers['supplier_lon'][i]
    receivers = df_small[(df_small['supplier_lat'] == this_lat) & (df_small['supplier_lon'] == this_lon)]
    
    #plot the receivers with a new color each time
    latlon = receivers['receiver']
    #print(receivers.shape[0])
    #print(receivers.shape[1])
    receiver_count += receivers.shape[0] #verify 1000
    
    #for each receiver  
    for j in range(0,len(latlon)): #coord in latlon: 
        
        lat = latlon.iloc[j][0]
        lon = latlon.iloc[j][1]
        
        volume = receivers.iloc[j]['volume']
        if volume > quantiles[3]:
            quantile = 1 #80-100 percentile
        elif volume > quantiles[2]:
            quantile = 0.8 #60-80 percentile
        elif volume > quantiles[1]:
            quantile = 0.6 #40-60 percentile
        elif volume > quantiles[0]:
            quantile = 0.4 #20-40 percentile
        else:
            quantile  = 0.2 #0-20 percentile
        line_weight = quantile * 3
        #print(line_weight)
        #print(volume, quantile)
        
        #line coordinates
        line_coords = [ [unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i]],
                      [lat, lon] ]#[coord[0], coord[1] ]  ]
        my_PolyLine= folium.PolyLine(locations=line_coords,color=palette[i],weight=line_weight) #color and supplier use i index
        mapit.add_child(my_PolyLine)
        
        #dot for the receiver
        folium.CircleMarker( location=[ lat,lon ], fill=True, fill_color=palette[i], color=palette[i],  radius=1 ).add_to( mapit )

print(receiver_count)
mapit

mapit.save(outfile= "visualization.html")

3961


,supplier_lat,supplier_lon
0,33.894790,-118.164740
1,33.910116,-118.221906
2,34.125000,-119.400300
3,33.854967,-118.336907
4,34.074052,-118.437913
...,...,...
274,32.876900,-117.166400
275,33.615700,-114.686500
276,33.243200,-115.498000
277,33.603440,-114.643390
